In [1]:
TYPE = 'bulbus'
# TYPE = 'atrium'
# TYPE = 'heart'

In [2]:
from __future__ import print_function
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os 
import glob
import skimage.io as io
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
import skimage.transform as trans
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as K
from pymodules.unet_model import unet,dice_coef
from pymodules.data import trainGenerator,testGenerator,labelVisualize
from tensorflow.keras.callbacks import  EarlyStopping, ReduceLROnPlateau 
import matplotlib.pyplot as plt

env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.


In [3]:
import numpy as np
import cv2
import os
from PIL import Image

X_test = []
for filepath in os.listdir('../data/test_images/ventral_samples_R0004'):
    image = Image.open('../data/test_images/ventral_samples_R0004/'+filepath)
    image = image.resize((256, 256))
    # convert image to numpy array
    data = np.asarray(image)
    data = data/255.
    X_test.append(data)
X_test = np.array(X_test)
Y_test = []
for filepath in os.listdir('../data/test_images/ventral_mask_%s_R0004'%TYPE):
    image = Image.open('../data/test_images/ventral_mask_%s_R0004/'%TYPE+filepath).convert('L')
    image = image.resize((256, 256))
    data = np.asarray(image).reshape(256,256,1)
    data = data/255.
    Y_test.append(data)
Y_test = np.array(Y_test)

In [4]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 10
epochs = 400
iterations_per_epoch = 300

data_gen_args = dict(rotation_range=0.3,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.1,
                    horizontal_flip=True,
                    fill_mode='nearest')

train_generator = trainGenerator(batch_size,'../data/train_images','ventral_samples','ventral_mask_%s'%TYPE,data_gen_args,image_color_mode='rgb',mask_color_mode = "grayscale")
val_datagen = ImageDataGenerator()

In [5]:
model = unet(input_size = (256,256,3))

In [ ]:
model_checkpoint = ModelCheckpoint('ventral.hdf5', monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor="val_loss",verbose = 1,mode='min',patience=70)
reduce_lr =  ReduceLROnPlateau(monitor = "val_loss", factor = 0.5, patience = 10,verbose = 0, mode = "auto", epsilon = 1e-04, cooldown = 0,min_lr = 1e-5)
model.fit_generator(train_generator,steps_per_epoch=iterations_per_epoch,epochs=epochs,callbacks=[model_checkpoint,early_stopping,reduce_lr],validation_data=val_datagen.flow(X_test, Y_test, batch_size=batch_size))

Instructions for updating:
Please use Model.fit, which supports generators.
Found 565 images belonging to 1 classes.
Found 565 images belonging to 1 classes.
Epoch 1/400
300/300 [==============================] - ETA: 0s - loss: 0.9999 - dice_coef: 2.2331e-04 - accuracy: 0.9825
Epoch 00001: val_loss improved from inf to 1.00000, saving model to ventral.hdf5
300/300 [==============================] - 140s 466ms/step - loss: 0.9999 - dice_coef: 2.2331e-04 - accuracy: 0.9825 - val_loss: 1.0000 - val_dice_coef: 0.0014 - val_accuracy: 0.9838
Epoch 2/400
300/300 [==============================] - ETA: 0s - loss: 1.0000 - dice_coef: 8.9673e-05 - accuracy: 0.9825
Epoch 00002: val_loss did not improve from 1.00000
300/300 [==============================] - 139s 464ms/step - loss: 1.0000 - dice_coef: 8.9673e-05 - accuracy: 0.9825 - val_loss: 1.0000 - val_dice_coef: 0.0013 - val_accuracy: 0.9838
Epoch 3/400
133/300 [============>.................] - ETA: 1:16 - loss: 1.0000 - dice_coef: 8.9466e-0